In [16]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd

In [89]:

load_dotenv()

api_key =  os.environ.get("riot_api_key")

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd

def create_session():
    session = requests.Session()

    # Configurar retries automáticos em caso de falhas
    retries = Retry(
        total=5,  # Número máximo de tentativas
        backoff_factor=1,  # Fator para calcular o tempo de espera (exponencial)
        status_forcelist=[429, 500, 502, 503, 504],  # Erros a serem tratados como falhas temporárias
        method_whitelist=["HEAD", "GET", "OPTIONS"],  # Métodos HTTP que irão passar pelo retry
        raise_on_status=False  # Não levanta exceção, apenas retorna o erro para tratá-lo
    )
    adapter = HTTPAdapter(max_retries=retries, pool_connections=50, pool_maxsize=50)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session

# Criar sessão compartilhada
session = create_session()

In [83]:
# FUNC
def chall_players(key):
    link = f"https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={key}"
    req = session.get(link)


    df = pd.DataFrame(req.json())

    normalize_df = pd.json_normalize(df['entries'])
    df = pd.concat([df.drop(columns='entries'), normalize_df], axis=1)
    df.drop(columns=["name","rank","leagueId"],axis=1,inplace=True)
    df.sort_values("leaguePoints").reset_index(drop=True,inplace=True)

    df_id = process_ids(api_key,df,"summonerId")
    df_id.rename(columns={'id': 'summonerId'}, inplace=True)
    resultado = pd.merge(df_id, df, on='summonerId', how='inner')
    return resultado

def add_puuid(key,id):
    link=f"https://br1.api.riotgames.com/lol/summoner/v4/summoners/{id}?api_key={key}"
    req = session.get(link)
    return req

def process_ids(api_key, df, column_name):
    """
    Percorre uma coluna de um DataFrame, faz as requisições para cada ID, e organiza os resultados em um DataFrame.
    """
    results = []
    i=0
    for summoner_id in df[column_name]:
        data = add_puuid(api_key, summoner_id).json()
        if data:
            results.append(data)  # Adiciona os dados válidos na lista de resultados
            
            print(data)
    # Transforma a lista de resultados em um DataFrame
    result_df = pd.DataFrame(results)
    return result_df

In [100]:
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor
import requests

session = requests.Session()

def add_puuid(key, id, session):
    """
    Função para adicionar os dados do invocador a partir do seu ID.
    """
    link = f"https://br1.api.riotgames.com/lol/summoner/v4/summoners/{id}?api_key={key}"
    req = session.get(link)
    return req.json()

def process_ids(api_key, df, column_name, reqs_per_second, session):
    """
    Processa uma coluna do DataFrame, fazendo requisições para cada ID simultaneamente
    e controlando o número de requisições por segundo.
    """
    results = []
    delay = 1 / reqs_per_second  # Intervalo entre os lotes de requisições

    def process_id(summoner_id):
        data = add_puuid(api_key, summoner_id, session)
        if data:
            results.append(data)
            print(data)

    with ThreadPoolExecutor(max_workers=reqs_per_second) as executor:
        # Para cada lote de requisições, dispara múltiplas simultaneamente
        for i in range(0, len(df), reqs_per_second):
            batch = df[column_name][i:i + reqs_per_second]  # Lote de requisições
            executor.map(process_id, batch)  # Faz as requisições simultâneas
            time.sleep(delay)  # Pausa para controlar a taxa de requisições

    # Transforma a lista de resultados em um DataFrame
    result_df = pd.DataFrame(results)
    return result_df

def chall_players(key, reqs_per_second):
    """
    Função que pega os jogadores de Challenger e os dados de invocadores.
    """
    link = f"https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={key}"
    req = session.get(link)

    df = pd.DataFrame(req.json())

    # Normaliza e organiza os dados
    normalize_df = pd.json_normalize(df['entries'])
    df = pd.concat([df.drop(columns='entries'), normalize_df], axis=1)
    df.drop(columns=["name", "rank", "leagueId"], axis=1, inplace=True)
    df.sort_values("leaguePoints").reset_index(drop=True, inplace=True)

    # Processa os IDs e os merge
    df_id = process_ids(key, df, "summonerId", reqs_per_second, session)
    df_id.rename(columns={'id': 'summonerId'}, inplace=True)
    resultado = pd.merge(df_id, df, on='summonerId', how='inner')
    
    return resultado


reqs_per_second = 10  # Exemplo: fazer 10 requisições simultâneas por vez



In [105]:
teste_vel = chall_players(api_key,2)


{'id': '69sqzslyKFgEfUHlKP9zy11_xiOoFJGfB__zZfwhHvZX', 'accountId': 'sh2QEp6BtIjvDv8UVf7OkGM7qA82TijQDZyARywn5uE', 'puuid': '4FRogKJxnPLAzhOl4ofD-PN7j-TjWC2LTfxx-i32XL6-qvtHTQGIbUmJ2DCD2IOQW71Huxy7IeuXRA', 'profileIconId': 536, 'revisionDate': 1736239528633, 'summonerLevel': 69}
{'id': 'fOrbu2n5DtE9qy28qUrgKBaD3jxphGSZx0cuB_xlAA-5cpdYxhY4cmQDvg', 'accountId': 'O48UBoLL6-qBmr-u8pFi69LKYm9jLLvQai5XWEX1KhI8B2XtZPJCcUaL', 'puuid': 's8qcLieY9RtxNGgJ7_cNHb7yM8Y8FJB8NBirXIGwdja2O6ZAIjDASVNBpXDBmeDsw7ZhrNqiwVy4dQ', 'profileIconId': 6766, 'revisionDate': 1735863360000, 'summonerLevel': 137}
{'id': 'NjlpNEr3hMF2AYDp2eyaIEYb-Wr4y4RBFLGncVk5nNn3', 'accountId': '30TbyJ1YjOMZ7qIUzhM2v5QvmBwgSO54_Gyn4NhHE0Y', 'puuid': '1xHcB3dGcWANeBntLEWlfxAHR9VPglTSf1qV_TquVnaleVjhdYso2U0Z8T6bHEsUCEQNdDppGEpQ7A', 'profileIconId': 6645, 'revisionDate': 1736212522320, 'summonerLevel': 942}
{'id': 'qSW5hRsT6AreMleBadsMhLE-x2jZf-CSeU5Vl-WQFbshx3s', 'accountId': 'aipFve6sMAghEcvPI_E_DrQxe42J4FJaxae7xRbfzcI5n00', 'puuid'

In [103]:
teste_vel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   summonerId     99 non-null     object 
 1   accountId      99 non-null     object 
 2   puuid          99 non-null     object 
 3   profileIconId  99 non-null     float64
 4   revisionDate   99 non-null     float64
 5   summonerLevel  99 non-null     float64
 6   status         0 non-null      object 
 7   tier           99 non-null     object 
 8   queue          99 non-null     object 
 9   leaguePoints   99 non-null     int64  
 10  wins           99 non-null     int64  
 11  losses         99 non-null     int64  
 12  veteran        99 non-null     bool   
 13  inactive       99 non-null     bool   
 14  freshBlood     99 non-null     bool   
 15  hotStreak      99 non-null     bool   
dtypes: bool(4), float64(3), int64(3), object(6)
memory usage: 9.8+ KB
